In [1]:
import json
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import re

In [25]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='sakila')

In [26]:
mycursor = cnx.cursor()

In [4]:
with open("bda-modulo-2-evaluacion-final-marianela-gomez/exercise_script.sql", "r", encoding="utf-8") as file:
    sql_script = file.read()

In [5]:
clean_queries = []
     
for query in sql_script.split(";"):
    query = query.strip()
    
    if '--' not in query:
        clean_queries.append(query)
    if '#' in query:
        query = re.sub(r'\s#+\s[A-Z]\s[A-Z]\s[A-Z]\s[A-Z]\s[A-Z]\s#+\s#+', '', query)
        query = re.sub(r'#', '', query)
        clean_queries.append(query)
    if '--' in query:
        query = re.sub(r'--.+', '', query.strip())
        clean_queries.append(query)
    if '/*' in query:
        query = re.sub(r'\s+\/\*\s.*\s.+[a-zA-Z]+.+\s+[a-zA-Z].+\s+[a-zA-Z].+\s+[a-zA-Z].+\s+[a-zA-Z].+\s+[a-zA-Z].+', '', query)
        clean_queries.append(query)
   

In [6]:
clean_queries

['USE sakila',
 '\n\nSELECT DISTINCT lower(title) AS film_title\n\tFROM film',
 '\n\nSELECT lower(title) AS film_title\n\tFROM film \n    WHERE rating = "PG-13"',
 '\n\nSELECT lower(title) AS film_title, lower(description)\n\tFROM film \n    WHERE description LIKE ("%amazing%")',
 '\n\nSELECT lower(title) AS film_title\n\tFROM film \n    WHERE length > 120',
 '\n\nSELECT lower(first_name) AS actor_name\n\tFROM actor',
 '\n\nSELECT CONCAT(lower(first_name), " ", lower(last_name)) AS actor_name\n\tFROM actor\n    WHERE last_name LIKE ("Gibson")',
 '\n    \nSELECT lower(first_name) AS actor_name\n\tFROM actor\n    WHERE actor_id BETWEEN 10 AND 20',
 '\n\nSELECT lower(title) AS film_title\n\tFROM film \n    WHERE rating NOT IN ("R", "PG-13")',
 '\n\nSELECT rating, COUNT(rating) AS num_of_films\n\tFROM film\n    GROUP BY rating',
 '\n\nSELECT CONCAT(lower(c.first_name), " ", lower(c.last_name)) AS customer_name, COUNT(c.customer_id) AS rented_films\n\tFROM customer AS c\n    INNER JOIN rent

In [8]:
for query in clean_queries:
    if "" == query:
        clean_queries.remove(query)
    if "--" in query:
        clean_queries.remove(query)
    if "/*" in query:
        clean_queries.remove(query)

In [9]:
len(clean_queries)

25

In [31]:
results = []
for query in clean_queries:
    mycursor.execute(query)
    results.append(mycursor.fetchall())

In [41]:
df = pd.DataFrame(results[-1])

In [43]:
df.sample(10)

,0,1,2
3852,tom miranda,matthew carrey,2
4733,mena temple,fay winslet,1
4208,parker goldberg,will wilson,1
3548,goldie brody,ellen presley,1
9047,daryl crawford,morgan williams,2
9681,emily dee,mena hopper,1
6599,scarlett damon,jon chase,3
847,joe swank,kirsten akroyd,1
9419,ewan gooding,jon chase,1
5639,angela hudson,harvey hope,1


In [46]:
df_2 = pd.read_sql_query(clean_queries[-1], cnx)

C:\Users\maria\AppData\Local\Temp\ipykernel_24612\1034347106.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql_query(clean_queries[-1], cnx)


In [47]:
df_2.sample(10)

,actor_1,actor_2,films_together
2258,cameron streep,groucho williams,1
1878,lucille tracy,minnie kilmer,1
7537,gene willis,renee tracy,1
1302,vivien bergen,whoopi hurt,2
4287,julia barrymore,harvey hope,1
3516,tom mckellen,mary keitel,1
186,ed chase,uma wood,3
1359,vivien bergen,gene mckellen,1
1558,fred costner,reese west,2
1223,uma wood,tim hackman,1
